# Assignment 4

\- Yash Gupta (190997)

I am giving you a dataset that contains 70 judgments a subject has made about the size of hypothetical people based on their weight (in kilos) and height (in inches). The subject has categorized people into three categories - small, average and large. 

The dataset data.csv contains the 70 actual judgments made by the subject as a 70x3 matrix. The first column contains weights, the second contains heights. The third column contains the category label assigned by the subject (small = 1, average = 2, large = 3). 

I am also giving you a test set test.csv of 10 more weight-height combinations as a 10x2 matrix (same column interpretations). I want you to tell me what a generalized context model would predict this subjects' category labels to be, assuming  
(i) he is polite, and is far more likely to call someone big average than large  
(ii) He is more likely to use weight than height to make category judgments about size.

### Q1. (30 points)

Implement a GCM encoding these assumptions and give me quantitative predictions on the test set. Submit both code and category responses for the data points. 

### Q2. (40 points)

I am also sharing with you, John McDonnell's python implementation of Anderson's Rational Model of Categorization (rational.py). Modify the code to obtain category predictions for the data I have shared with you. 

In [1]:
# Implementation of Anderson's venerable "rational" model of categorization.
# Assumes that stimuli were generated by a mixture of Gaussian distributions;
# rather than compute the full Bayesian posterior, it views items sequentially
# and assigns each to the maximum a posteriori cluster.
#
# At the end it is presented with a stimulus with one item missing, and
# predicts the probability that its value is a '0' or a '1'.
#
# Implemented in python by John McDonnell
#
# References: Anderson (1990) and Anderson (1991),

import numpy as np
from random import shuffle

#Utility functions:

class dLocalMAP:
    """
    See Anderson (1990, 1991)
    'Categories' renamed 'clusters' to avoid confusion.
    Discrete version.
    
    Stimulus format is a list of integers from 0 to n-1 where n is the number
    of possible features (e.g. [1,0,1])
    
    args: c, alphas
    """
    
    def __init__(self, args):
        self.partition = [[]]
        self.c, self.alpha = args
        self.alpha0 = sum(self.alpha.T)
        self.N = 0
    
    def probClustVal(self, k, i, val):
        """Find P(j|k)"""
        cj = len([x for x in self.partition[k] if x[i]==val])
        nk = len(self.partition)
        return (cj + self.alpha[i][val])/(nk + self.alpha0[i])
    
    def condclusterprob(self, stim, k):
        """Find P(F|k)"""
        pjks = []
        for i in range(len(stim)):
            cj = len([x for x in self.partition[k] if x[i]==stim[i]])
            nk = len(self.partition[k])
            pjks.append( (cj + self.alpha[i][stim[i]])/(nk + self.alpha0[i]) )
        return np.product( pjks )
        
    
    def posterior(self, stim):
        """Find P(k|F) for each cluster"""
        pk = np.zeros( len(self.partition) )
        pFk = np.zeros( len(self.partition) )
        
        # existing clusters:
        for k in range(len(self.partition)):
            pk[k] = self.c * len(self.partition[k])/ ((1-self.c) + self.c * self.N)
            if len(self.partition[k])==0: # case of new cluster
                pk[k] = (1-self.c) / (( 1-self.c ) + self.c * self.N)
            pFk[k] = self.condclusterprob( stim, k)
        
        # put it together
        pkF = (pk*pFk) # / sum( pk*pFk )
        
        return pkF
    
    def stimulate(self, stim):
        """Argmax of P(k|F) + P(0|F)"""
        winner = np.argmax( self.posterior(stim) )
        print("Stim: ", stim)
        print("Partition: ", self.partition)
        print(self.posterior(stim))
        
        if len(self.partition[winner]) == 0:
            self.partition.append( [] )
        self.partition[winner].append(stim)
        
        self.N += 1
    
    def query(self, stimulus):
        """Queried value should be -1."""
        qdim = -1
        for i in range(len(stimulus)):
            if stimulus[i] < 0:
                if qdim != -1:
                    raise Exception("ERROR: Multiple dimensions queried.")
                qdim = i
        
        self.N = sum([len(x) for x in self.partition])
        
        pkF = self.posterior(stimulus)
        pkF = pkF[:-1] / sum(pkF[:-1]) # eliminate `new cluster' prob
        
        pjF = np.array( [sum( [ pkF[k] * self.probClustVal(k, qdim, j) \
                for k in range(len(self.partition)-1)] ) 
                for j in range(len( self.alpha[qdim] ))] )
        
        return pjF / sum(pjF)

def testlocalmapD():
    """
    Tests the Anderson's ratinal model using the Medin & Schaffer (1978) data.
    
    This script will print out the probability that each item belongs to each
    of the existing clusters or to a new cluster, and the model assign it to
    the most likely cluster. To see that the model is working correctly, you
    can follow along with Anderson (1991), which steps through in the same way.
    """
    stims = [[1, 1, 1, 1, 1], # Medin & Schaffer (1978)
             [1, 0, 1, 0, 1], 
             [1, 0, 1, 1, 0],
             [0, 0, 0, 0, 0],
             [0, 1, 0, 1, 1],  
             [0, 1, 0, 0, 0]]
    # These are the classic Shepard Type II and Type IV datasets.
    # Uncomment the one you want to try out; you might want to uncomment
    # shuffling the stims too if you don't care about order.
    #stims = [[0, 0, 0, 0], [0, 0, 1, 0], [1, 1, 0, 1], [1, 1, 1, 1], [1, 0, 0, 0], [1, 0, 1, 1], [0, 1, 0, 0], [0, 1, 1, 1]] # Type IV
    stims = [[0, 0, 0, 0], [0, 0, 1, 0], [1, 1, 0, 0], [1, 1, 1, 0], [1, 0, 0, 1], [1, 0, 1, 1], [0, 1, 0, 1], [0, 1, 1, 1]] # Type II
    
    for _ in range(1):
        model = dLocalMAP([.5, np.ones((len(stims[0]),2))])
        
        shuffle(stims)
        for s in stims:
            model.stimulate(s)
        print(model.partition)
        
        print("Prob vals for 0,0,0,0,?", model.query( [0]*(len(stims[0])-1) + [-1] ))


### Q3. (30 points)

For both GCM and RMC, show empirically using the dataset I've shared that both models assume exchangeability of data, viz. the order in which data enters the model does not affect the category labels of the model for any given subset of data. 